In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import requests
import gmaps


# Import API key
from config import gkey

In [2]:
#load data
df = pd.read_csv("cities.csv")
df.rename(columns = {"Unnamed: 0":"city"}, inplace = True)
df.dropna(inplace =  True)
df

,city,country,Lat,Lng,temp (f),humidity %,cloudiness %,wind (mph)
0,willowmore,ZA,-33.29,23.49,65.95,55,0,9.75
1,marawi,SD,18.48,31.82,68.14,26,0,13.33
2,attert,BE,49.75,5.79,37.99,96,0,3.51
3,berlevag,NO,70.86,29.09,35.60,82,0,23.04
4,busselton,AU,-33.65,115.33,54.00,97,23,4.94
...,...,...,...,...,...,...,...,...
536,petropavlovsk-kamchatskiy,RU,53.05,158.65,28.40,74,1,4.47
537,tadine,NC,-21.55,167.88,77.79,73,100,7.11
538,west wendover,US,40.74,-114.07,39.20,41,1,8.05
539,kudahuvadhoo,MV,2.67,72.89,82.54,72,100,6.55


In [3]:
#configure gmaps
gmaps.configure(api_key = gkey)
#use lat and lng as locations and humidity as the weight
locations = df[["Lat", "Lng"]]
humidity = df["humidity %"]
#add heatmap to map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                 dissipating = False)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
#configure heatmap settings
heat_layer.max_intensity = 125
heat_layer.point_radius = 3

In [5]:
#trim dataframe based on conditions
ideal_df = df.loc[(df["humidity %"] < 25), :]
ideal_df = ideal_df.loc[(ideal_df["wind (mph)"] < 10), :]
ideal_df = ideal_df.loc[(ideal_df["temp (f)"] < 90) & (ideal_df["temp (f)"] > 60), :]
ideal_df = ideal_df.loc[(ideal_df["cloudiness %"] < 25), :]
#add a hotel column
ideal_df["hotel"] = ""
ideal_df

,city,country,Lat,Lng,temp (f),humidity %,cloudiness %,wind (mph),hotel
201,silver city,US,32.77,-108.28,60.80,21,1,8.05,
430,roebourne,AU,-20.78,117.13,83.71,19,0,6.24,
455,djibo,BF,14.10,-1.63,77.36,18,0,8.59,
478,karratha,AU,-20.74,116.85,81.25,20,0,8.30,
495,tessalit,ML,20.20,1.01,69.84,23,0,9.06,
496,arlit,NE,18.74,7.39,71.11,23,0,8.95,


In [6]:
#prep api url and params
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location" : "",
    "radius" : 5000,
    "type" : "lodging",
    "keyword" : "hotel",
    "key" : gkey
}
#loop through df making api calls to find hotels near these cities
for i, row in ideal_df.iterrows():
    params["location"] = f"{row['Lat']},{row['Lng']}"
    response = requests.get(base_url, params=params).json()
    #if no results were returned, print that fact and move drop row
    if len(response["results"]) <= 0:
        ideal_df.drop(index = i, inplace = True)
        print(f"No known hotels near {row['city']}. Dropping from dataframe.")
        #move on to next iterations
        continue
    #add the hotel name to the dataframe
    ideal_df.loc[i, "hotel"] = response["results"][0]["name"]
ideal_df

No known hotels near tessalit. Dropping from dataframe.


,city,country,Lat,Lng,temp (f),humidity %,cloudiness %,wind (mph),hotel
201,silver city,US,32.77,-108.28,60.80,21,1,8.05,Murray Hotel
430,roebourne,AU,-20.78,117.13,83.71,19,0,6.24,Latitude20 Roebourne Village
455,djibo,BF,14.10,-1.63,77.36,18,0,8.59,Hôtel Le Berger
478,karratha,AU,-20.74,116.85,81.25,20,0,8.30,Karratha International Hotel
496,arlit,NE,18.74,7.39,71.11,23,0,8.95,Hôtel Telwa Bungalow


In [7]:
#prep lists to send to marker layer
marker_locations = list(zip(ideal_df["Lat"], ideal_df["Lng"]))
hotel_info = list(zip(ideal_df["hotel"], ideal_df["city"], ideal_df["country"]))
#get hotel info as strings for hover text and as html for info box text
hotel_info_strings = []
hotel_info_html = []
for hotel in hotel_info:
    hotel_info_strings.append(f"Hotel: {hotel[0]}\nCity: {hotel[1].title()}\nCountry: {hotel[2]}")
    hotel_info_html.append(f"""
    <dl>
    <dt><b>Name: </b></dt><dd>{hotel[0]}</dd>
    <dt><b>City: </b></dt><dd>{hotel[1].title()}</dd>
    <dt><b>Country: </b></dt><dd>{hotel[2]}</dd>
    </dl>
    """)
#add marker layer
marker_layer = gmaps.marker_layer(marker_locations, 
                                  hover_text = hotel_info_strings, 
                                  info_box_content = hotel_info_html)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))